In [11]:
import os
import sys
import autogen
import pandas as pd
from typing import Dict
import pulp
import json
import logging
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [12]:
# Define the LLM configuration
llm_config = {
    "model": "gpt-4o-mini",
    "api_key": os.environ.get("OPENAI_API_KEY"),
    "temperature": 0,
    "seed": 42,
}

# User Input Agent
user_input_agent = autogen.AssistantAgent(
    name="UserInputAgent",
    llm_config=llm_config,
    system_message="""You are an agent that processes user requests for fantasy football lineups.
    Parse the user's input and convert it into a structured format for lineup optimization.
    If the input is empty or doesn't contain specific constraints, return an empty dictionary.
    Use the following examples as a guide:

    Input: "I want Justin Jefferson in my lineup"
    Output: {"must_include": ["justinjefferson"]}

    Input: "Can you make sure to prioritize running backs?"
    Output: {"position_emphasis": {"RB": 2.0}}

    Input: "I'd like to focus on players from the Vikings"
    Output: {"team_preference": "MIN"}

    Input: "Generate me 2 unique lineups"
    Output: {"num_lineups": 2}

    Input: "Create 3 lineups with different strategies"
    Output: {"num_lineups": 3, "diverse_strategies": true}

    Input: "" (empty input)
    Output: {}

    Provide your output as a valid JSON object."""
)

# User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0
)

class LineupOptimizerAgent(autogen.AssistantAgent):
    def __init__(self, name, data_file, llm_config):
        super().__init__(name=name, llm_config=llm_config)
        self.data = pd.read_csv(data_file)
        self.preprocess_data()
        self.previous_lineups = []

    def preprocess_data(self):
        # Replace spaces with underscores in player names
        self.data['player_name'] = self.data['player_name'].str.replace(' ', '_')
        
        # Convert salary to float
        self.data['salary'] = self.data['salary'].replace('[\\$,]', '', regex=True).astype(float)
        
        # Convert projected points to float
        self.data['projected_points'] = pd.to_numeric(self.data['projected_points'], errors='coerce')
        
        # Print some information about the data
        print(f"Total number of players: {len(self.data)}")
        print(f"Unique positions: {self.data['player_position_id'].unique()}")
        print(f"Salary range: ${self.data['salary'].min()} - ${self.data['salary'].max()}")
        print(f"Projected points range: {self.data['projected_points'].min()} - {self.data['projected_points'].max()}")
        print("Sample of processed player names:")
        print(self.data['player_name'].head())

    def optimize_lineup(self, constraints: Dict) -> Dict:
        players = self.data.to_dict('records')
        prob = pulp.LpProblem("Fantasy Football", pulp.LpMaximize)
        
        player_vars = pulp.LpVariable.dicts("players", 
                                            ((p['player_name'], p['player_position_id']) for p in players), 
                                            cat='Binary')

        # Objective: Maximize total projected points
        prob += pulp.lpSum([player['projected_points'] * player_vars[player['player_name'], player['player_position_id']] 
                            for player in players])

        # Basic constraints
        self.add_basic_constraints(prob, player_vars, players)
        
        # Apply user-defined constraints
        self.apply_user_constraints(prob, player_vars, players, constraints)

        # Ensure diversity from previous lineups
        self.ensure_lineup_diversity(prob, player_vars, players)

        # Solve the problem
        prob.solve()

        if pulp.LpStatus[prob.status] != 'Optimal':
            return {'error': f"Optimization failed: {pulp.LpStatus[prob.status]}"}

        return self.build_lineup(player_vars, players)

    def add_basic_constraints(self, prob, player_vars, players):
        prob += pulp.lpSum([player['salary'] * player_vars[player['player_name'], player['player_position_id']] for player in players]) <= 50000
        prob += pulp.lpSum([player_vars[p['player_name'], 'QB'] for p in players if p['player_position_id'] == 'QB']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], 'RB'] for p in players if p['player_position_id'] == 'RB']) >= 2
        prob += pulp.lpSum([player_vars[p['player_name'], 'WR'] for p in players if p['player_position_id'] == 'WR']) >= 3
        prob += pulp.lpSum([player_vars[p['player_name'], 'TE'] for p in players if p['player_position_id'] == 'TE']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], 'DST'] for p in players if p['player_position_id'] == 'DST']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] for p in players]) == 9

    def apply_user_constraints(self, prob, player_vars, players, constraints):
        if 'must_include' in constraints:
            for player_name in constraints['must_include']:
                prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                    for p in players if p['player_name'].lower() == player_name.lower()]) == 1

        if 'position_emphasis' in constraints:
            for position, emphasis in constraints['position_emphasis'].items():
                prob += pulp.lpSum([player['projected_points'] * player_vars[p['player_name'], position] 
                                    for p in players if p['player_position_id'] == position]) >= emphasis * prob.objective

        if 'team_preference' in constraints:
            team = constraints['team_preference']
            prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                for p in players if p['player_team_id'] == team]) >= 3

    def ensure_lineup_diversity(self, prob, player_vars, players):
        for prev_lineup in self.previous_lineups:
            prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                for p in prev_lineup.values() if p is not None]) <= 6  # Allow up to 6 players to overlap

    def build_lineup(self, player_vars, players):
        lineup = {
            'QB': None, 'RB1': None, 'RB2': None, 'WR1': None, 'WR2': None, 'WR3': None, 
            'TE': None, 'FLEX': None, 'DST': None
        }

        for player in players:
            if player_vars[player['player_name'], player['player_position_id']].value() == 1:
                position = player['player_position_id']
                if position == 'QB' and lineup['QB'] is None:
                    lineup['QB'] = player
                elif position == 'RB':
                    if lineup['RB1'] is None:
                        lineup['RB1'] = player
                    elif lineup['RB2'] is None:
                        lineup['RB2'] = player
                    elif lineup['FLEX'] is None:
                        lineup['FLEX'] = player
                elif position == 'WR':
                    if lineup['WR1'] is None:
                        lineup['WR1'] = player
                    elif lineup['WR2'] is None:
                        lineup['WR2'] = player
                    elif lineup['WR3'] is None:
                        lineup['WR3'] = player
                    elif lineup['FLEX'] is None:
                        lineup['FLEX'] = player
                elif position == 'TE':
                    if lineup['TE'] is None:
                        lineup['TE'] = player
                    elif lineup['FLEX'] is None:
                        lineup['FLEX'] = player
                elif position == 'DST':
                    lineup['DST'] = player

        self.previous_lineups.append(lineup)
        return lineup

    def reset_lineups(self):
        self.previous_lineups = []

    def format_lineup(self, lineup):
        output = "Optimized Lineup:\n"
        total_salary = 0
        total_points = 0
        for position, player in lineup.items():
            if player:
                output += f"{position}: {player['player_name']} (Projected: {player['projected_points']:.2f}, Salary: ${player['salary']})\n"
                total_salary += player['salary']
                total_points += player['projected_points']
            else:
                output += f"{position}: Not filled\n"
        output += f"\nTotal Salary: ${total_salary}"
        output += f"\nTotal Projected Points: {total_points:.2f}"
        return output
    

optimizer_agent = LineupOptimizerAgent(
    name="LineupOptimizer",
    data_file='data/merged_fantasy_football_data.csv',
    llm_config=llm_config
)


def display_lineup(lineup):
    html = """
    <style>
        .lineup-table {
            border-collapse: collapse;
            width: 100%;
        }
        .lineup-table th, .lineup-table td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }
        .lineup-table tr:nth-child(even) {
            background-color: #f2f2f2;
        }
        .lineup-table th {
            background-color: #4CAF50;
            color: white;
        }
    </style>
    <table class="lineup-table">
        <tr>
            <th>Position</th>
            <th>Player</th>
            <th>Projected Points</th>
            <th>Salary</th>
        </tr>
    """
    total_salary = 0
    total_points = 0
    for position, player in lineup.items():
        if player:
            html += f"""
            <tr>
                <td>{position}</td>
                <td>{player['player_name']}</td>
                <td>{player['projected_points']:.2f}</td>
                <td>${player['salary']}</td>
            </tr>
            """
            total_salary += player['salary']
            total_points += player['projected_points']
    html += f"""
        <tr>
            <th colspan="2">Totals</th>
            <th>{total_points:.2f}</th>
            <th>${total_salary}</th>
        </tr>
    </table>
    """
    display(HTML(html))

def create_fantasy_football_ui(optimizer_agent, user_input_agent, user_proxy):
    # Input widget
    input_text = widgets.Text(
        value='',
        placeholder='Enter your lineup request',
        description='Request:',
        disabled=False
    )

    # Generate button
    generate_button = widgets.Button(
        description='Generate Lineup',
        disabled=False,
        button_style='success',
        tooltip='Click to generate lineup'
    )

    # Output widget
    output = widgets.Output()

    # Feedback buttons
    feedback_buttons = widgets.ToggleButtons(
        options=['Good', 'Bad'],
        description='Feedback:',
        disabled=False,
        button_style='info'
    )

    # Variable to store the last generated lineup
    last_lineup = None

    def on_generate_button_clicked(b):
        nonlocal last_lineup
        with output:
            clear_output()
            print("Generating lineup...")
            constraints = {}
            if input_text.value:
                user_proxy.send(
                    input_text.value,
                    user_input_agent,
                    request_reply=True
                )
                user_input_response = user_input_agent.last_message()['content']
                print(f"User Input Agent response: {user_input_response}")
                
                try:
                    constraints = json.loads(user_input_response)
                except json.JSONDecodeError:
                    print("Error processing input. Using default settings.")

            lineup = optimizer_agent.optimize_lineup(constraints)
            if 'error' in lineup:
                print(f"Error generating lineup: {lineup['error']}")
                feedback_buttons.disabled = True
            else:
                display_lineup(lineup)
                feedback_buttons.disabled = False
                last_lineup = lineup  # Store the last generated lineup

    generate_button.on_click(on_generate_button_clicked)

    def on_feedback_given(change):
        if change['type'] == 'change' and change['name'] == 'value':
            with output:
                print(f"Feedback recorded: {change['new']}")
                if last_lineup:
                    optimizer_agent.add_feedback(last_lineup, change['new'].lower())
                else:
                    print("No lineup to provide feedback on.")
                feedback_buttons.disabled = True

    feedback_buttons.observe(on_feedback_given)

    # Layout
    ui = widgets.VBox([
        input_text, 
        generate_button, 
        output,
        feedback_buttons
    ])
    
    display(ui)

IndentationError: expected an indented block after class definition on line 47 (2434771058.py, line 48)

In [ ]:
create_fantasy_football_ui(optimizer_agent, user_input_agent, user_proxy)